# Embedding Chunks

Once the chunks were created and stored during the parsing step of the RAG pipeline, now its time to embed this chunks into a vector based on an embedding model, and then store these vectors into a vector DB.

## Import libraries

In [1]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct, Filter, FieldCondition, MatchValue
from sentence_transformers import SentenceTransformer
import json
import sys
import uuid

sys.path.append("..")

from utils.gcp.gcs import get_file
from rag_llm_energy_expert.config import GCP_CONFIG
from rag_llm_energy_expert.credentials import get_qdrant_config
from rag_llm_energy_expert.ingest_document import upload_document, parse_file, create_points

c:\Users\Emmanuel\Desktop\git_repos\RAG-LLM-Energy-Expert\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Embeddings

Once the PDF text has been parsed, chunked, and stored in GCS, I'll use that chunks to embed the data into a vector database, I will be using [*Qdrant DB*](https://qdrant.tech/). The first thing to do is establish the embedding model to use and load the chunks stored in GCS

For this time, we'll be using the [*sentence-transformers/all-MiniLM-L6-v2*](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2) model:

- Number of Parameters: 22.7M (small)
- Embedding Dimension: 384
- Max tokens: 256

In [2]:
# load the qdrant config values
qdrant_config = get_qdrant_config()
gcp_config = GCP_CONFIG()

In [6]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"

# Name of the txt file where the chunks are stored
chunks_file = "chunks/resumen_reforma_energetica_256_2025-04-06.txt"

In [7]:
chunks = json.loads(get_file(chunks_file, bucket_name = gcp_config.BUCKET_NAME))

In [8]:
len(chunks)

309

Then, we wil load the embedding model from sentence-transformers

In [9]:
model = SentenceTransformer(model_name)

Create a vector for each chunk

In [10]:
for chunk_id, chunk_data in chunks.items():

    # Create the vector id
    vector_id = f"{uuid.uuid4()}"

    chunk_data["vector_id"] = vector_id

    # Embed the data into a vector
    chunk_data["vector"] = model.encode(chunk_data["text"])

Create a list of chunk's data

In [11]:
chunks_data = [chunks[f"chunk{x}"] for x in range(len(chunks))]

In [12]:
chunks_data[0]

{'Header 1': 'Resumen Ejecutivo',
 'data': '# Resumen Ejecutivo  \n-----  \n-----',
 'upload_date': '2025-04-06',
 'text': '# Resumen Ejecutivo\n\n\n-----\n\n-----\n\n## I. Introducción\n\nLa Reforma Energética es un paso decidido rumbo a la modernización del sector energético de\nnuestro país, sin privatizar las empresas públicas dedicadas a la producción y al aprovechamiento de los hidrocarburos y de la electricidad. La Reforma Energética, tanto constitucional como a\nnivel legistlación secundarias, surge del estudio y valoración de las distintas iniciativas presentadas por los partidos políticos representados en el Congreso.\n\n### La Reforma Energética tiene los siguientes objetivos y premisas fundamentales:\n\n1. Mantener la propiedad de la Nación sobre los hidrocarburos que se encuentran en el subsuelo.\n2. Modernizar y fortalecer, sin privatizar, a Petróleos Mexicanos (Pemex) y a la Comisión Federal de Electricidad (CFE) como Empresas Productivas del Estado, 100% públicas y 100%

In [13]:
vector_size = len(chunks_data[0]["vector"])

vector_size

384

Connect to the Qdrant Vector DB

In [14]:
qdrant_client = QdrantClient(url = qdrant_config.URL, api_key = qdrant_config.API_KEY.get_secret_value())

Create a new collection

In [15]:
collection_name = "energy_expert_v1"

In [16]:
if not qdrant_client.collection_exists(collection_name):
    qdrant_client.create_collection(
        collection_name = collection_name,
        vectors_config = VectorParams(size = vector_size, distance = Distance.DOT),
    )

else:
    print("Collection already exists")


Collection already exists


Generate a list of PointStruct objects

In [17]:
points = [PointStruct(
                    id = chunk_data["vector_id"],
                    vector = chunk_data["vector"],
                    payload = { key: value for key, value in chunk_data.items() if key not in ["vector_id", "vector"]}  
                       ) for chunk_data in chunks_data]

len(points)

309

In [18]:
points_title = points[0].payload["title"]

In [28]:
# Get the document's title
chunks_title = points[0]

# Finding if the document is already in the collection

# Create a filter to match payload
title_filter = Filter(
    must=[
        FieldCondition(
            key="title",
            match=MatchValue(value=points_title)
        )
    ]
)

# Scroll through all matching vectors
scroll_result = qdrant_client.scroll(
    collection_name=collection_name,
    scroll_filter=title_filter,
    limit=1 # In this case, I only need 1 vector to know if the document is already indexed
)

# Access the points
vectors = scroll_result[0]  # List of PointStruct

if len(vectors) > 0:
    print("The document that is being trying to index is already in the collection.")

else:
    print("Indexing document")
    qdrant_client.upsert(
        collection_name = collection_name,
        wait = True, 
        points = points,
    )

The document that is being trying to index is already in the collection.


## Indexing a PDF into a vector DB

This section shows the whole steps to load a PDF into a vector DB. The PDF file is stored in GCS

In [3]:
gcs_file_path = f"gs://{gcp_config.BUCKET_NAME}/documents/summaries/resumen_reforma_energetica.pdf"

In [4]:
chunks = parse_file(gcs_file_path,
                    chunk_size=256,
                    chunk_overlap=0)

2025-04-08 12:15:38.929 | INFO     | rag_llm_energy_expert.ingest_document:parse_file:41 - Parsing file...
2025-04-08 12:15:38.933 | INFO     | rag_llm_energy_expert.parsers_auxiliars:extract_pdf_content:32 - Extracting PDF content...
2025-04-08 12:15:38.935 | INFO     | rag_llm_energy_expert.parsers_auxiliars:extract_pdf_content:48 - google cloud storage path detected
2025-04-08 12:15:40.200 | INFO     | rag_llm_energy_expert.parsers_auxiliars:extract_pdf_content:86 - Converting PDF content to a markdown format...
2025-04-08 12:16:03.353 | INFO     | rag_llm_energy_expert.parsers_auxiliars:extract_pdf_content:102 - PDF content successfully extracted
2025-04-08 12:16:03.353 | INFO     | rag_llm_energy_expert.parsers_auxiliars:chunk_by_md_headers:119 - Chunking the text by markdown headers...
2025-04-08 12:16:03.369 | INFO     | rag_llm_energy_expert.parsers_auxiliars:chunk_by_md_headers:141 - Chunks based on markdown headers successfully created
2025-04-08 12:16:03.369 | INFO     | rag

In [7]:
points=create_points(chunks=chunks, embedding_model=SentenceTransformer(qdrant_config.EMBEDDING_MODEL))

2025-04-08 12:16:34.750 | INFO     | rag_llm_energy_expert.ingest_document:create_points:83 - Creating points from chunks file...
2025-04-08 12:16:34.754 | INFO     | rag_llm_energy_expert.ingest_document:create_points:95 - Embedding chunks...
2025-04-08 12:16:42.155 | INFO     | rag_llm_energy_expert.ingest_document:create_points:114 - Embeddings created


In [8]:
points[0]

PointStruct(id='4056b616-187f-497a-a6b1-ff4e804b91d2', vector=[-0.08761797845363617, 0.07058341056108475, 0.013568367809057236, -0.007789564784616232, -0.0325322300195694, 0.06386642158031464, 0.02928394265472889, 0.07830828428268433, -0.02154732309281826, -0.04089831933379173, 0.046019986271858215, -0.014016227796673775, -0.11006162315607071, -0.024974362924695015, -0.050907693803310394, -0.05500343441963196, -0.04999767616391182, 0.03196965157985687, 0.0260700061917305, 0.04113233461976051, 0.06488081067800522, 0.025954028591513634, 0.006191559601575136, 0.0009083159966394305, -0.029840033501386642, -0.017980486154556274, 0.02115805447101593, -0.007033641915768385, 0.011362326331436634, -0.07433632016181946, 0.03645186498761177, -0.010294289328157902, 0.04713594168424606, -0.013658802025020123, 0.0075075747445225716, 0.06611251085996628, -0.05643364414572716, 0.008608045987784863, 0.007552890572696924, 0.02792053297162056, -0.08371800929307938, -0.14766690135002136, -0.08526746183633

In [9]:
#  Previous execution time: 2 min, actual time: 1 min 11 s, only using batch embeddings
upload_document(
    file_path=gcs_file_path,
    chunk_overlap=qdrant_config.CHUNK_OVERLAP,
    embedding_model=qdrant_config.EMBEDDING_MODEL,
    vectordb_collection=qdrant_config.COLLECTION_NAME + qdrant_config.COLLECTION_VERSION 
    )

2025-04-08 12:16:52.629 | INFO     | rag_llm_energy_expert.ingest_document:upload_document:146 - embedding model successfully initialized
2025-04-08 12:16:52.632 | INFO     | rag_llm_energy_expert.ingest_document:parse_file:41 - Parsing file...
2025-04-08 12:16:52.633 | INFO     | rag_llm_energy_expert.parsers_auxiliars:extract_pdf_content:32 - Extracting PDF content...
2025-04-08 12:16:52.636 | INFO     | rag_llm_energy_expert.parsers_auxiliars:extract_pdf_content:48 - google cloud storage path detected
2025-04-08 12:16:52.983 | INFO     | rag_llm_energy_expert.parsers_auxiliars:extract_pdf_content:86 - Converting PDF content to a markdown format...
2025-04-08 12:17:14.716 | INFO     | rag_llm_energy_expert.parsers_auxiliars:extract_pdf_content:102 - PDF content successfully extracted
2025-04-08 12:17:14.716 | INFO     | rag_llm_energy_expert.parsers_auxiliars:chunk_by_md_headers:119 - Chunking the text by markdown headers...
2025-04-08 12:17:14.737 | INFO     | rag_llm_energy_expert.